In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import linear_model
import statsmodels.api as sm
import datetime as dt

In [26]:
df=pd.read_csv('Datasource/Cases Report.csv')
df.head()

,Outcome,Age,Gender,Reporting PHU,SpecimenDate,CaseReported Date,PHUCity,TestReported Date,CaseAcquisition info,AccurateEpisode Dt,PHU Address,PHU Website,OutbreakRelated,PHU Latitude,PHU Longitude,PHU Postal Code
0,Resolved,80s,FEMALE,Toronto Public Health,9/24/2020,2/6/2021,Toronto,2/6/2021,OB,1/26/2021,"277 Victoria Street, 5th Floor",www.toronto.ca/community-people/health-wellnes...,Yes,43.656591,-79.379358,M5B 1W2
1,Resolved,<20,MALE,Peel Public Health,12/24/2020,2/1/2021,Mississauga,2/1/2021,CC,1/29/2021,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
2,Resolved,20s,MALE,Lambton Public Health,9/23/2020,1/15/2021,Point Edward,1/14/2021,CC,1/10/2021,160 Exmouth Street,www.lambtonhealth.on.ca,Yes,42.986416,-82.404808,N7T 7Z6
3,Resolved,20s,MALE,"Region of Waterloo, Public Health",9/20/2020,12/29/2020,Waterloo,9/23/2020,NO KNOWN EPI LINK,9/20/2020,99 Regina Street South,www.regionofwaterloo.ca,NaN,43.462876,-80.520913,N2J 4V3
4,Resolved,20s,FEMALE,Ottawa Public Health,9/21/2020,12/24/2020,Ottawa,9/22/2020,CC,9/21/2020,100 Constellation Drive,www.ottawapublichealth.ca,NaN,45.345665,-75.763912,K2G 6J8


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28500 entries, 0 to 28499
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Outcome               28500 non-null  object 
 1   Age                   28500 non-null  object 
 2   Gender                28500 non-null  object 
 3   Reporting PHU         28500 non-null  object 
 4   SpecimenDate          28292 non-null  object 
 5   CaseReported Date     28500 non-null  object 
 6   PHUCity               28500 non-null  object 
 7   TestReported Date     28082 non-null  object 
 8   CaseAcquisition info  28500 non-null  object 
 9   AccurateEpisode Dt    28500 non-null  object 
 10  PHU Address           28500 non-null  object 
 11  PHU Website           28500 non-null  object 
 12  OutbreakRelated       8710 non-null   object 
 13  PHU Latitude          28500 non-null  float64
 14  PHU Longitude         28500 non-null  float64
 15  PHU Postal Code    

In [28]:
df.isnull().sum()

Outcome                     0
Age                         0
Gender                      0
Reporting PHU               0
SpecimenDate              208
CaseReported Date           0
PHUCity                     0
TestReported Date         418
CaseAcquisition info        0
AccurateEpisode Dt          0
PHU Address                 0
PHU Website                 0
OutbreakRelated         19790
PHU Latitude                0
PHU Longitude               0
PHU Postal Code             0
dtype: int64

# Outcome, Age, Gender

In [29]:
df['Outcome'].value_counts()

Resolved        27192
Fatal            1307
Not Resolved        1
Name: Outcome, dtype: int64

In [30]:
df['Age'].value_counts()

20s    5899
30s    4363
50s    4280
40s    3952
60s    2879
<20    2556
80s    1737
70s    1699
90+    1135
Name: Age, dtype: int64

In [31]:
df['Gender'].value_counts()

FEMALE            14470
MALE              14006
UNSPECIFIED          21
GENDER DIVERSE        3
Name: Gender, dtype: int64

In [32]:
df['Gender']=df['Gender'].str.lower()

# SpecimenDate, TestReported Date

In [33]:
df['SpecimenDate'].value_counts()

4/15/2020     404
4/16/2020     389
4/13/2020     383
4/11/2020     356
4/17/2020     336
             ... 
2/9/2021        1
10/16/2020      1
10/15/2020      1
3/3/2020        1
12/15/2020      1
Name: SpecimenDate, Length: 242, dtype: int64

In [34]:
df['SpecimenDate'] =  pd.to_datetime(df['SpecimenDate'], format='%m/%d/%Y')

In [35]:
df['SpecimenDate']

0       2020-09-24
1       2020-12-24
2       2020-09-23
3       2020-09-20
4       2020-09-21
           ...    
28495   2020-02-29
28496   2020-02-28
28497   2020-02-28
28498   2020-02-28
28499   2020-02-29
Name: SpecimenDate, Length: 28500, dtype: datetime64[ns]

In [36]:
df['SpecimenDate'] = df['SpecimenDate'].fillna(df.groupby('Reporting PHU')['SpecimenDate'].transform('median'))

In [37]:
df['TestReported Date'] =  pd.to_datetime(df['TestReported Date'], format='%m/%d/%Y')
df['TestReported Date'] = df['TestReported Date'].fillna(df.groupby('Reporting PHU')['TestReported Date'].transform('median'))

In [38]:
df.isnull().sum()

Outcome                     0
Age                         0
Gender                      0
Reporting PHU               0
SpecimenDate                0
CaseReported Date           0
PHUCity                     0
TestReported Date           0
CaseAcquisition info        0
AccurateEpisode Dt          0
PHU Address                 0
PHU Website                 0
OutbreakRelated         19790
PHU Latitude                0
PHU Longitude               0
PHU Postal Code             0
dtype: int64

In [39]:
df['CaseReported Date'] =  pd.to_datetime(df['CaseReported Date'], format='%m/%d/%Y')
df['AccurateEpisode Dt'] =  pd.to_datetime(df['AccurateEpisode Dt'], format='%m/%d/%Y')

# OutbreakRelated

In [40]:
df['OutbreakRelated'].unique()

array(['Yes', nan], dtype=object)

In [41]:
bool_series = pd.isnull(df['OutbreakRelated'])
df[bool_series].groupby('CaseAcquisition info')['CaseAcquisition info'].count().sort_values(ascending=True).reset_index(name='Count')

,CaseAcquisition info,Count
0,UNSPECIFIED EPI LINK,130
1,OB,246
2,MISSING INFORMATION,250
3,TRAVEL,1721
4,NO KNOWN EPI LINK,6293
5,CC,11150


In [42]:
bool_series = pd.notnull(df['OutbreakRelated'])
df[bool_series].groupby('CaseAcquisition info')['CaseAcquisition info'].count().sort_values(ascending=True).reset_index(name='Count')

,CaseAcquisition info,Count
0,TRAVEL,2
1,UNSPECIFIED EPI LINK,3
2,MISSING INFORMATION,4
3,NO KNOWN EPI LINK,88
4,CC,393
5,OB,8220


In [43]:
pd.options.mode.chained_assignment = 'warn'
df.loc[df['CaseAcquisition info']=='OB','OutbreakRelated'] = df.loc[df['CaseAcquisition info']=='OB','OutbreakRelated'].fillna('Yes')

In [44]:
df[df['CaseAcquisition info']=='OB']['OutbreakRelated'].unique()

array(['Yes'], dtype=object)

In [45]:
df['OutbreakRelated'] = df['OutbreakRelated'].fillna('No')

In [46]:
df.isnull().sum()

Outcome                 0
Age                     0
Gender                  0
Reporting PHU           0
SpecimenDate            0
CaseReported Date       0
PHUCity                 0
TestReported Date       0
CaseAcquisition info    0
AccurateEpisode Dt      0
PHU Address             0
PHU Website             0
OutbreakRelated         0
PHU Latitude            0
PHU Longitude           0
PHU Postal Code         0
dtype: int64

In [48]:
df.columns

Index(['Outcome', 'Age', 'Gender', 'Reporting PHU', 'SpecimenDate',
       'CaseReported Date', 'PHUCity', 'TestReported Date',
       'CaseAcquisition info', 'AccurateEpisode Dt', 'PHU Address',
       'PHU Website', 'OutbreakRelated', 'PHU Latitude', 'PHU Longitude',
       'PHU Postal Code'],
      dtype='object')

In [49]:
df.columns=['Outcome', 'Age', 'Gender', 'ReportingPHU', 'SpecimenDate',
       'CaseReportedDate', 'PHUCity', 'TestReportedDate',
       'CaseAcquisitionInfo', 'AccurateEpisodeDt', 'PHUAddress',
       'PHUWebsite', 'OutbreakRelated', 'PHULatitude', 'PHULongitude',
       'PHUPostalCode']
df.to_csv('CasesReport.csv',index=False,sep=';',date_format = '%m/%d/%Y')